In [8]:
# Importing nescessary modules
import os
import re
import shutil
import numpy as np
import pandas as pd


from processing.utils import metafile_handler as mfh
from processing.utils import file_handler as fh
from utils import dtw

# from benchmarks import frechet

In [9]:
DATA_ROME = "dataset/rome/output/"
TEST_SET_ROME = "dataset/rome/output/META-50.txt"
FULL_SET_ROME = "dataset/rome/output/META-1000.txt"
SIMILARITIES_OUTPUT_FOLDER_ROME = "benchmarks/similarities/rome/"
DTW_FILENAME_ROME = "rome-dtw-fullset.csv"
DTW_FILENAME_TEST_ROME = "rome-dtw-testset.csv"

DATA_PORTO = "dataset/porto/output/"
TEST_SET_PORTO = "dataset/porto/output/META-50.txt"
FULL_SET_PORTO = "dataset/porto/output/META-1000.txt"
SIMILARITIES_OUTPUT_FOLDER_PORTO = "benchmarks/similarities/porto/"
DTW_FILENAME_PORTO = "porto-dtw-fullset.csv"
DTW_FILENAME_TEST_PORTO = "porto-dtw-testset.csv"

DATA_KOLUMBUS = "dataset/kolumbus/output/"
TEST_SET_KOLUMBUS = "dataset/kolumbus/output/META-50.txt"
FULL_SET_KOLUMBUS = "dataset/kolumbus/output/META-1000.txt"
SIMILARITIES_OUTPUT_FOLDER_KOLUMBUS = "benchmarks/similarities/kolumbus/"
DTW_FILENAME_KOLUMBUS = "kolumbus-dtw-fullset.csv"
DTW_FILENAME_TEST_KOLUMBUS = "kolumbus-dtw-testset.csv"

In [17]:
def deleteFile(file_name: str, folder_name: str) -> None:
    file_path = os.path.join(folder_name, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print("Failed to remove %s. Reason: %s" % (file_path, e))


def romeSet(file_size: int) -> str:
    return f"dataset/rome/output/META-{file_size}.txt"

def portoSet(file_size: int) -> str:
    return f"dataset/porto/output/META-{file_size}.txt"


In [11]:
# Using Cython DTW, to speed things up


def generate_dtw_similarities(
    data_folder: str, meta_file: str, file_name: str, similarities_output_folder: str
):
    deleteFile(file_name, similarities_output_folder)

    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    df = dtw.cy_dtw(trajectories)

    df.to_csv(os.path.join(similarities_output_folder, file_name))


def generate_parallell_dtw_similarities(
    data_folder: str, meta_file: str, file_name: str, similarities_output_folder: str
):
    deleteFile(file_name, similarities_output_folder)

    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    df = dtw.cy_dtw_pool(trajectories)
    df.to_csv(os.path.join(similarities_output_folder, file_name))

In [6]:
# DTW similarities for Rome

# Test set
# generate_dtw_similarities(
#     DATA_ROME, TEST_SET_ROME, DTW_FILENAME_TEST_ROME, SIMILARITIES_OUTPUT_FOLDER_ROME
# )
generate_parallell_dtw_similarities(
    DATA_ROME, TEST_SET_ROME, DTW_FILENAME_TEST_ROME, SIMILARITIES_OUTPUT_FOLDER_ROME
)

# Full set
# generate_dtw_similarities(DATA_ROME, FULL_SET_ROME, DTW_FILENAME_ROME)
generate_parallell_dtw_similarities(
    DATA_ROME, FULL_SET_ROME, DTW_FILENAME_ROME, SIMILARITIES_OUTPUT_FOLDER_ROME
)

In [12]:
# DTW similarities for Porto
# Test set
# generate_dtw_similarities(
#     DATA_PORTO,
#     TEST_SET_PORTO,
#     DTW_FILENAME_TEST_PORTO,
#     SIMILARITIES_OUTPUT_FOLDER_PORTO,
# )
generate_parallell_dtw_similarities(
    DATA_PORTO,
    TEST_SET_PORTO,
    DTW_FILENAME_TEST_PORTO,
    SIMILARITIES_OUTPUT_FOLDER_PORTO,
)

# Full set
# generate_dtw_similarities(
#     DATA_PORTO, FULL_SET_PORTO, DTW_FILENAME_PORTO, SIMILARITIES_OUTPUT_FOLDER_PORTO
# )
generate_parallell_dtw_similarities(
    DATA_PORTO, FULL_SET_PORTO, DTW_FILENAME_PORTO, SIMILARITIES_OUTPUT_FOLDER_PORTO
)

In [8]:
# DTW similarities for KOLUMBUS
# Test set
# generate_dtw_similarities(DATA_KOLUMBUS, TEST_SET_KOLUMBUS, DTW_FILENAME_TEST_KOLUMBUS, SIMILARITIES_OUTPUT_FOLDER_KOLUMBUS)
generate_parallell_dtw_similarities(
    DATA_KOLUMBUS,
    TEST_SET_KOLUMBUS,
    DTW_FILENAME_TEST_KOLUMBUS,
    SIMILARITIES_OUTPUT_FOLDER_KOLUMBUS,
)

# Full set
# generate_dtw_similarities(
#     DATA_PORTO, FULL_SET_PORTO, DTW_FILENAME_PORTO, SIMILARITIES_OUTPUT_FOLDER_PORTO
# )
# generate_dtw_similarities(DATA_KOLUMBUS, FULL_SET_KOLUMBUS, DTW_FILENAME_KOLUMBUS, SIMILARITIES_OUTPUT_FOLDER_KOLUMBUS)
generate_parallell_dtw_similarities(
    DATA_KOLUMBUS,
    FULL_SET_KOLUMBUS,
    DTW_FILENAME_KOLUMBUS,
    SIMILARITIES_OUTPUT_FOLDER_KOLUMBUS,
)

In [13]:
#Time Efficiency
from multiprocessing import Pool

# Using Python, for measuring computation time fairly against the hash computations

sim = {
    "dtw_py" : dtw.measure_py_dtw,
    # "frechet_py" : frechet.measure_py_frechet,
    "dtw_cy" : dtw.measure_cy_dtw,
    # "frechet_cy" : frechet.measure_cy_frechet
}

def measure_similarities(measure: str, data_folder: str, meta_file: str, parallell_jobs: int = 10):
    """ Common method for measuring the efficiency of the similarity algorithms """
    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    with Pool() as pool:
        result = pool.map(sim[measure], [[trajectories, 1, 1] for _ in range(10)])
    return result

In [18]:
measure_similarities("dtw_py", DATA_PORTO, portoSet(50))

[[9.056223],
 [8.561636],
 [8.652308],
 [8.630231],
 [8.674166],
 [8.592920000000001],
 [8.515234999999999],
 [8.507482000000001],
 [8.480445],
 [8.80237]]

In [ ]:
measure_similarities("dtw_cy", DATA_PORTO, portoSet(1000))

In [ ]:
# Measuring the computation times of true similarities

runs = 10
data_sets = range(500,501,100)
output_folder = "../code/experiments/timing/"
file_name = "similarity_runtimes_true_dtw_rome_500.csv"

df = pd.DataFrame( index=[f"run_{x+1}" for x in range(runs)], columns=[x for x in data_sets])

for size in data_sets:
    print(f"Computing size {size}", end="\r")
    execution_times = measure_similarities("dtw_py", DATA_ROME, romeSet(size), parallell_jobs=runs)
    df[size] = [element[0] for element in execution_times]

df.to_csv(os.path.join(output_folder, file_name))
df
of = pd.read_csv("C:\\Users\\bafla\\Desktop\\Projects\\master\\code\\experiments\\timing\\similarity_runtimes_true_dtw_rome.csv", index_col=0)
f1 = pd.read_csv("C:\\Users\\bafla\\Desktop\\Projects\\master\\code\\experiments\\timing\\similarity_runtimes_true_dtw_rome_500.csv", index_col=0)

of["500"] = f1["500"]
#of["700"] = f1["700"]

of.to_csv("C:\\Users\\bafla\\Desktop\\Projects\\master\\code\\experiments\\timing\\similarity_runtimes_true_dtw_rome.csv")
20